In [2]:
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from itertools import combinations, permutations
from molvs import standardize_smiles

In [2]:
def pairwise(given_list):
    '''
    Generate all possible pairs based on a given list.
    s -> (s0,s1), (s1,s2), (s2, s3), ...
    '''
    res = list(permutations(given_list, 2))
    return res

In [3]:
def make_chunks(given_list, n_blocks_per_container):
    '''
    Generate all possible pairs based on a given list.
    s -> (s0,s1), (s1,s2), (s2, s3), ...
    '''
    res = list(permutations(given_list, n_blocks_per_container))
    return res

In [4]:
def deprotect_fmoc(target_smile):
    FMOC_smirks = '[#6:2][N:1]C(=O)OCC1c2ccccc2-c2ccccc12>>[#6:2][N:1]'
    fmoc_rxn = AllChem.ReactionFromSmarts(FMOC_smirks)
    reactant = Chem.MolFromSmiles(target_smile)
    products = fmoc_rxn.RunReactants((reactant,))
    try:
        deprotected_mol = products[0][0]
        deprotected_smile = Chem.MolToSmiles(deprotected_mol)
    except:
        deprotected_smile = target_smile
    return deprotected_smile

In [5]:
def amide_coupling(smile1, smile2):
    deprotect_first_smile  = deprotect_fmoc(smile1)
    deprotect_second_smile = deprotect_fmoc(smile2)
    mol1 = Chem.MolFromSmiles(deprotect_first_smile)
    mol2 = Chem.MolFromSmiles(deprotect_second_smile)
    smarts = "[C:1](=[O:2])O.[N:3] >> [C:1](=[O:2])[N:3]"
    rxn3 = AllChem.ReactionFromSmarts (smarts)
    products = rxn3.RunReactants ([mol1, mol2])
    resulting_smile_list = []
    try:
        for i in range(len(products)):
            resulting_smile = Chem.MolToSmiles(products[i][0])
            resulting_smile_list.append(resulting_smile)
    except:
        pass
    return resulting_smile_list


In [6]:
def multiple_amide_coupling(list_of_smiles):
    first_smile = list_of_smiles[0]
    for i in range(len(list_of_smiles)-1):
        try:
            first_smile = amide_coupling(first_smile, list_of_smiles[i+1])[0]
        except:
            first_smile = ''
    return first_smile

def make_libary(possible_smile_blocks):
    enumerated_library    = []
    for smiles_to_piece in possible_smile_blocks:
        resulting_cpd = multiple_amide_coupling(list(smiles_to_piece))
        try:
            canonicalized_resulting_smile = standardize_smiles(resulting_cpd)
            if canonicalized_resulting_smile not in enumerated_library:
                enumerated_library.append(canonicalized_resulting_smile)
        except:
            pass
    return enumerated_library

In [7]:
s1 = "O=C(O)CCC1CCN(C(=O)OCC2c3ccccc3-c3ccccc32)CC1"
s2 = "CN(CCCCC(=O)O)C(=O)OCC1c2ccccc2-c2ccccc21"
s3 = "O=C(NC1CCN(c2ccc(C(=O)O)cn2)CC1)OCC1c2ccccc2-c2ccccc21"
s4 = "O=C(NC1CCC(C(=O)O)CC1)OCC1c2ccccc2-c2ccccc21"
s5 = "O=C(O)Cn1c(-c2ccccc2)cc2ccccc21"
s6 = "N#CCn1nccc1C(=O)O"


list_of_smiles = [s1,s2,s3,s4,s5,s6]
desired_blocks_per_molecule = 2
possible_smile_blocks = make_chunks(list_of_smiles, desired_blocks_per_molecule)

make_libary(possible_smile_blocks)
    
# print(possible_smile_blocks)
# enumerate_smiles(list_of_smiles)

['CN(CCCCC(=O)O)C(=O)CCC1CCNCC1',
 'O=C(CCC1CCNCC1)NC1CCN(c2ccc(C(=O)O)cn2)CC1',
 'O=C(CCC1CCNCC1)NC1CCC(C(=O)O)CC1',
 'CNCCCCC(=O)N1CCC(CCC(=O)O)CC1',
 'CNCCCCC(=O)NC1CCN(c2ccc(C(=O)O)cn2)CC1',
 'CNCCCCC(=O)NC1CCC(C(=O)O)CC1',
 'NC1CCN(c2ccc(C(=O)N3CCC(CCC(=O)O)CC3)cn2)CC1',
 'CN(CCCCC(=O)O)C(=O)c1ccc(N2CCC(N)CC2)nc1',
 'NC1CCN(c2ccc(C(=O)NC3CCC(C(=O)O)CC3)cn2)CC1',
 'NC1CCC(C(=O)N2CCC(CCC(=O)O)CC2)CC1',
 'CN(CCCCC(=O)O)C(=O)C1CCC(N)CC1',
 'NC1CCC(C(=O)NC2CCN(c3ccc(C(=O)O)cn3)CC2)CC1',
 'O=C(O)CCC1CCN(C(=O)Cn2c(-c3ccccc3)cc3ccccc32)CC1',
 'CN(CCCCC(=O)O)C(=O)Cn1c(-c2ccccc2)cc2ccccc21',
 'O=C(Cn1c(-c2ccccc2)cc2ccccc21)NC1CCN(c2ccc(C(=O)O)cn2)CC1',
 'O=C(Cn1c(-c2ccccc2)cc2ccccc21)NC1CCC(C(=O)O)CC1',
 'N#CCn1nccc1C(=O)N1CCC(CCC(=O)O)CC1',
 'CN(CCCCC(=O)O)C(=O)c1ccnn1CC#N',
 'N#CCn1nccc1C(=O)NC1CCN(c2ccc(C(=O)O)cn2)CC1',
 'N#CCn1nccc1C(=O)NC1CCC(C(=O)O)CC1']

In [8]:
# def draw_reaction(reactants, products):
#     rxn = AllChem.ReactionFromSmarts(reactants+'>>'+products, useSmiles=True)
#     return Draw.ReactionToImage(rxn)

# deprotect_first_smile         = 'O=C(O)CCC1CCNCC1'
# deprotect_second_smile        = 'CNCCCCC(=O)O'
# canonicalized_resulting_smile = 'CN(CCCCC(=O)O)C(=O)CCC1CCNCC1'
# reactants = deprotect_first_smile+'.'+deprotect_second_smile
# products  = canonicalized_resulting_smile
# draw_reaction(reactants, products)